# Lab 1: Implementing self-editing memory from scratch

## Preparation
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; 
<b>Note: </b>The Letta libraries have been progressing since the filming of this course and you should be careful to use the requirements.txt file to load the revisions that are compatible with these notebooks if you download them and run them in your own evironment.
</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

## Section 0: Setup OpenAI 

In [ ]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=openai_api_key
)

## Section 1: Breaking down the LLM context window
### A simple agent's context window

In [ ]:
model = "gpt-4o-mini"

In [ ]:
system_prompt = "You are a chatbot."

In [ ]:
# Make the completion request with the tool usage
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt}, 
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"}, 
    ]
)
chat_completion.choices[0].message.content

### Adding memory to the context 


In [ ]:
agent_memory = {"human": "Name: Bob"}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

In [ ]:
import json


chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt + "[MEMORY]\n" + \
         json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "What is my name?"},
    ],
)
chat_completion.choices[0].message.content

## Section 2: Modifing the memory with tools 

### Defining a memory editing tool 


In [ ]:
agent_memory = {"human": "", "agent": ""}

def core_memory_save(section: str, memory: str): 
    agent_memory[section] += '\n' 
    agent_memory[section] += memory 

In [ ]:
agent_memory

In [ ]:
core_memory_save("human", "The human's name is Charles")

In [ ]:
agent_memory

In [ ]:
# tool description 
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool 
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",            
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

In [ ]:
agent_memory = {"human": ""}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "My name is Bob"},
    ],
    # tool schemas 
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

### Executing the tool 


In [ ]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

In [ ]:
# run the function with the specified arguments 
core_memory_save(**arguments)

In [ ]:
agent_memory

### Running the next agent step 


In [ ]:
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "what is my name"},
    ],
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response.message

## Implementing an agentic loop


In [ ]:
agent_memory = {"human": ""}

In [ ]:
system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool." 

In [ ]:
def agent_step(user_message): 
    
    # prefix messages with system prompt and memory
    messages = [
        # system prompt 
        {"role": "system", "content": system_prompt_os}, 
        # memory
        {
            "role": "system", 
            "content": "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ]    

    # append the most recent message
    messages.append({"role": "user", "content": user_message})
    
    # agentic loop 
    while True: 
        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[core_memory_save_metadata]
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response 
        messages.append(response.message)
        
        # if NOT calling a tool (responding to the user), return 
        if not response.message.tool_calls: 
            return response.message.content

        # if calling a tool, execute the tool
        else: 
            print("TOOL CALL:", response.message.tool_calls[0].function)
            
            # parse the arguments from the LLM function call
            arguments = json.loads(
                response.message.tool_calls[0].function.arguments
            )

            # run the function with the specified arguments
            core_memory_save(**arguments)

            # add the tool call response to the message history 
            messages.append({
                "role": "tool", 
                "tool_call_id": response.message.tool_calls[0].id, 
                "name": "core_memory_save", 
                "content": f"Updated memory: {json.dumps(agent_memory)}"
            })

In [ ]:
agent_step("my name is bob.")

In [ ]:
# Try some prompts of your own!